### Voor- en nadelen van LINQ
- `+` SQL syntax check door Python --> fouten ontdekt bij het ontwikkelen
- `+` Porteerbaar tussen databanksystemen
- `-` Geen integratie met Python-objecten.
- `-` Extra syntax moet aangeleerd worden. 

In [7]:
from sqlalchemy import create_engine  
engine = create_engine('mssql+pyodbc://localhost\\SQL2025/xtreme?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

conn = engine.connect() 

In [8]:
# Setting up MetaData with Table Objects
from sqlalchemy import MetaData, Table, Column, Integer, String, Float, Numeric
metadata = MetaData()   # This object is essentially a facade around a Python dictionary 
                        # that stores a series 
                        # of Table objects keyed to their string name. 

# Option 1: explicit Table objects
Product = Table('Product', metadata, 
               Column('ProductID', Integer, primary_key=True),
               Column('ProductName', String),
               Column('Color', String),
               Column('Sizes', String),
               Column('M_F', String),
               Column('Price', Numeric(8,2)),
               Column('ProductTypeID', Integer),
               Column('ProductClassID', Integer),
               Column('SupplierID', Integer),
               Column('ReorderLevel', Integer),
               Column('UnitsInStock', Integer))          

# Option 2: reflecting tables: generate Table objects automatically from database     
OrdersDetail = Table('OrdersDetail', metadata, autoload_with=engine)
ProductType = Table('ProductType', metadata, autoload_with=engine)
Courier = Table('Courier', metadata, autoload_with=engine)  

### Select

```sql
select top 5 productid, productname, unitsinstock
from product
order by unitsinstock desc;
```

In [ ]:
from sqlalchemy import select
stmt = select(Product.c.ProductID,Product.c.ProductName, Product.c.UnitsInStock) \
       .order_by(Product.c.UnitsInStock.desc()) \
       .limit(5)

rows = conn.execute(stmt)

**Complexere SELECT** 
   
```sql
select top 10 p.ProductID, p.ProductName, 
sum(od.Quantity*od.UnitPrice) as sales
from Product p join OrdersDetail od
on p.ProductID=od.ProductID
group by p.ProductID, p.ProductName
order by sum(od.Quantity*od.UnitPrice) desc;
```

In [ ]:
from sqlalchemy import select, func

stmt = select(Product.c.ProductID,Product.c.ProductName,func.sum(OrdersDetail.c.Quantity * OrdersDetail.c.UnitPrice)) \
       .select_from(Product) \
       .join(OrdersDetail, Product.c.ProductID == OrdersDetail.c.ProductID) \
       .group_by(Product.c.ProductID, Product.c.ProductName) \
       .order_by(func.sum(OrdersDetail.c.Quantity * OrdersDetail.c.UnitPrice).desc()) \
       .limit(10)

print(stmt)

rows = conn.execute(stmt)

**UPDATE** \
Verhoog de prijzen van alle producten van het producttype x met y % (x en y zijn parameters). 

   ```sql
   update Product 
   set Price = Price * 1.10
   where ProductTypeID 
   in (select ProductTypeID 
   from Producttype where producttypename='Mountain')
   ``` 

In [ ]:
from sqlalchemy import update,bindparam

subq = select(ProductType.c.ProductTypeID).where(ProductType.c.ProductTypeName == bindparam("type"))

stmt = (update(Product).values(Price=Product.c.Price * (1 + bindparam("pct"))).where(Product.c.ProductTypeID.in_(subq)))
print(stmt) # string representation of the statement
conn.execute(stmt,{"pct":0.1, "type":"Mountain"}) 

conn.rollback()
conn.commit()

**INSERT**  
    ```sql
   insert into Courier (CourierID, CourierName,Website)
   values (11,'bpost','www.bpost.be')
   ```

In [ ]:
from sqlalchemy import insert
stmt = insert(Courier).values(CourierID=11,CourierName='bpost', Website='www.bpost.be')
conn.execute(stmt)

conn.rollback()
conn.commit()

**DELETE**
   ```sql
   delete from Courier where CourierName = 'bpost'
    ```

In [ ]:
from sqlalchemy import delete
stmt = delete(Courier).where(Courier.c.CourierName == 'bpost')
conn.execute(stmt)

conn.rollback()
conn.commit()

In [ ]:
conn.close()